## Fixed End Forces
This module computes the fixed end forces (moments and shears) due to transverse loads
acting on a 2-D planar structural member.

In [29]:
from __future__ import division, print_function

import numpy as np
import sys
sys.path[:0] = ['../../../']
from salib import *

In [20]:
class FEF(object):
    
    def __init__(self,f0=0.,f1=0.,f2=0.,f3=0.,f4=0.,f5=0.):
        if type(f0) is np.ndarray:
            self.fefs = f0.copy()
        else:
            self.fefs = np.array([f0,f1,f2,f3,f4,f5])
        
    @property
    def f0(self):
        return self.fefs[0]
    
    def __getitem__(self,ix):
        return self.fefs[ix]
    
    def __add__(self,other):
        assert type(self) is type(other)
        new = FEF(self.fefs+other.fefs)
        return new
    
    def __repr__(self):
        return '{}({},{},{},{},{},{})'.format(self.__class__.__name__,*(list(self.fefs)))

In [23]:
##test
f = FEF(1,2,0,4)
f

FEF(1.0,2.0,0.0,4.0,0.0,0.0)

In [24]:
##test
g = f+f+f
g

FEF(3.0,6.0,0.0,12.0,0.0,0.0)

In [25]:
##test
f[1]

2.0

In [16]:
##test
g[np.ix_([3,0,1])]

array([ 0.,  3.,  6.])

In [26]:
##test
g[([3,0,1],)]

array([ 12.,   3.,   6.])

In [28]:
##test
f0,f1,f2,f3,f4,f5 = g
f3

12.0

In [18]:
class MemberLoad(object):
    
    def fefs(self):
        """Return the complete set of 6 fixed end forces produced by the load."""
        raise NotImplementedError()
        
    def shear(self,x):
        """Return the shear force that is in equilibrium with that
        produced by the portion of the load to the left of the point at 
        distance 'x'.  'x' may be a scalar or a 1-dimensional array
        of values."""
        raise NotImplementedError()
        
    def moment(self,x):
        """Return the bending moment that is in equilibrium with that
        produced by the portion of the load to the left of the point at 
        distance 'x'.  'x' may be a scalar or a 1-dimensional array
        of values."""
        raise NotImplementedError()

In [19]:
@extend
class MemberLoad:
    
    @property
    def vpts(self):
        """Return an array of points (x-distances from end-0) at which the
        shear force must be evaluated in order to draw a proper shear force
        diagram."""
        raise NotImplementedError()
    
    @property
    def mpts(self):
        """Return an array of points (x-distances from end-0) at which the
        bending moment must be evaluated in order to draw a proper bending
        moment diagram."""
        raise NotImplementedError()

In [32]:
class PL(MemberLoad):
    
    def __init__(self,L,W1,A):
        self.L = L
        self.P = W1
        self.A = A
        
    def fefs(self):
        P = self.P
        L = self.L
        a = self.A
        b = L-a
        m2 = -P*a*b*b/(L*L)
        m5 = P*a*a*b/(L*L)
        v1 = (m2 + m5 - P*b)/L
        v4 = -(m2 + m5 + P*a)/L
        return FEF(0.,v1,m2,0.,v4,m5)
    
    def shear(self,x):
        return -self.P*(x>self.A)
    
    def moment(self,x):
        return self.P*(x-self.A)*(x>self.A)
        
    def __repr__(self):
        return '{}(L={},W1={},A={})'.format(self.__class__.__name__,self.L,self.P,self.A)

In [35]:
#test:
p = PL(1000.,300.,400.)
p

PL(L=1000.0,W1=300.0,A=400.0)

In [36]:
##test:
p.fefs()

FEF(0.0,-194.4,-43200.0,0.0,-105.6,28800.0)

In [37]:
@extend
class MemberLoad:
    
    EPSILON = 1.0E-6

@extend
class PL:
    
    @property
    def vpts(self):
        return (self.A-self.EPSILON,self.A+self.EPSILON,0)
    
    @property
    def mpts(self):
        return (None,self.A,1)

In [39]:
p = PL(1000.,300.,400.)
p.vpts

TypeError: object() takes no parameters

In [60]:
g

FEF(3.0,6.0,0.0,0.0,0.0,0.0)